In [8]:
import numpy as np
import pandas as pd
import plotly.express as px


class Person:
    def __init__(self, name: str, foresight: int):
        self.name = name
        self.current_life_score = 0
        self.foresight = foresight

    def initialise_credit_bank(self, n_credits: int, delay: int):
        self.credit_bank = {
            f"credit_{i + 1}": [0] * delay * (i + 1) for i in range(n_credits)
        }

    def calc_new_life_score(
        self, decision_number: int, choice_array: np.ndarray
    ) -> float:
        # Get the row index of the highest sum of values in the choice array up to the person's "foresight" index
        highest_value_index = np.argmax(
            np.sum(choice_array[:, : self.foresight], axis=1)
        )

        # Extract this row from the choice array
        highest_value_row = choice_array[highest_value_index, :]

        # Add the first value of this row to the person's life score
        self.current_life_score += highest_value_row[0]

        # Update the person's credit bank with other values from this row
        for credit_number, credit_list in self.credit_bank.items():
            credit_number_int = int(credit_number.split("_")[-1])  # <-- use as index
            credit_list.append(highest_value_row[credit_number_int])

        # Add credit to person's life score
        for credit_number, credit_list in self.credit_bank.items():
            self.current_life_score += credit_list[decision_number]

        return self.current_life_score


class LifeSimulation:
    def __init__(
        self,
        people: list[Person],
        alpha: float,
        n_choices: int,
        n_credits: int,
        delay: int,
    ):
        self.people = people
        self.alpha = alpha
        self.n_choices = n_choices
        self.n_credits = n_credits
        self.delay = delay

    def scale_credit_scores(self, credit_scores: np.ndarray) -> np.ndarray:
        indices = np.arange(len(credit_scores))
        return credit_scores * np.exp(self.alpha * indices)

    def add_person(self, person: Person):
        self.people.append(person)

    def get_people(self) -> list[Person]:
        return self.people

    def get_person_by_name(self, name: str) -> Person | None:
        for person in self.people:
            if person.name == name:
                return person
        return None

    def get_number_of_people(self) -> int:
        return len(self.people)

    def generate_choice_array(self, n_choices: int, n_credits: int) -> np.ndarray:
        choice_list = []
        for i in range(n_choices):
            choice_list.append(
                self.scale_credit_scores(
                    np.random.uniform(low=-1.0, high=1.0, size=(n_credits + 1,))
                )
            )
        return np.array(choice_list)

    def simlulate_life(self, number_of_decisions: int):
        life_score_history = {"decision_number": list[int](range(number_of_decisions))}
        for person in self.people:
            person.initialise_credit_bank(self.n_credits, self.delay)
            life_score_history[person.name] = []
            for decision_number in range(number_of_decisions):
                choice_array = self.generate_choice_array(
                    self.n_choices, self.n_credits
                )
                person.calc_new_life_score(decision_number, choice_array)
                life_score_history[person.name].append(person.current_life_score)
        return pd.DataFrame(life_score_history)


def main():
    # Define people
    people = [
        Person("Amy", 1),
        Person("Barry", 2),
        Person("Cindy", 3),
        Person("Danny", 4),
    ]

    # Define simulation configuration
    alpha = 0.3
    n_choices = 10
    n_credits = 4
    delay = 100
    number_of_decisions = 500

    # Run simulation
    simulation = LifeSimulation(people, alpha, n_choices, n_credits, delay)
    life_score_history_df = simulation.simlulate_life(number_of_decisions)
    print(life_score_history_df)

    fig = px.line(
        life_score_history_df,
        x="decision_number",
        y=life_score_history_df.columns,
        title="Life Score History",
    )
    fig.show()


if __name__ == "__main__":
    main()

     decision_number         Amy       Barry       Cindy       Danny
0                  0    0.613595    0.780920    0.992197    0.882269
1                  1    1.498370    1.166824    1.107506    1.762442
2                  2    2.390294    1.692086    1.569627    2.563718
3                  3    3.370450    2.374978    1.896611    3.301231
4                  4    4.282598    2.545332    1.805760    3.986012
..               ...         ...         ...         ...         ...
495              495  445.633611  571.905416  775.143046  846.801230
496              496  442.852148  573.425021  779.001530  852.584714
497              497  444.453085  574.523306  778.876662  853.170866
498              498  441.775960  575.405073  783.640819  853.842101
499              499  443.561848  571.146348  782.491235  861.255425

[500 rows x 5 columns]
